In [9]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [11]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [12]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Santa Catarina - Produção de Cimento (t)', axis=1)
data

,Ano,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda,Santa Catarina - Consumo de Cimento (t)
0,2008,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,0.802185,0.868622,5.483335e+08,2.546644e+09,...,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,0.764435,0.777491,2653.587000
1,2009,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,0.802866,0.870023,5.302764e+08,2.679945e+09,...,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,0.766524,0.776296,2669.400000
2,2010,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,0.803485,0.870869,5.046425e+08,2.891930e+09,...,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,0.768517,0.775508,2852.759000
3,2011,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,0.804044,0.871172,4.854622e+08,3.091988e+09,...,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,0.770407,0.775118,3087.810000
4,2012,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,0.804247,0.870803,4.867661e+08,3.189506e+09,...,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,0.771704,0.774917,3268.310000
5,2013,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,0.805268,0.870203,5.225848e+08,3.093872e+09,...,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,0.774346,0.775826,3466.227000
6,2014,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,0.805624,0.868970,5.676413e+08,2.779340e+09,...,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.775791,0.776479,3246.749286
7,2015,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,0.805509,0.867217,5.179392e+08,2.561731e+09,...,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.776440,0.777102,3017.084229
8,2016,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,0.807604,0.875108,4.524144e+08,2.516859e+09,...,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.782298,0.769848,2776.217486
9,2017,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,5.526597,0.807023,0.872664,4.213689e+08,2.497710e+09,...,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.782053,0.770470,2891.489000


In [13]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,0.802185,0.868622,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,0.764435,0.777491
1,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,0.802866,0.870023,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,0.766524,0.776296
2,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,0.803485,0.870869,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,0.768517,0.775508
3,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,0.804044,0.871172,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,0.770407,0.775118
4,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,0.804247,0.870803,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,0.771704,0.774917
5,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,0.805268,0.870203,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,0.774346,0.775826
6,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,0.805624,0.868970,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.775791,0.776479
7,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,0.805509,0.867217,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.776440,0.777102
8,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,0.807604,0.875108,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.782298,0.769848
9,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,5.526597,0.807023,0.872664,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.782053,0.770470


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3466.227000
5     3246.749286
6     3017.084229
7     2776.217486
8     2891.489000
9     2976.485000
10    3258.027000
11    3537.032000
12    4089.333000
13    4255.808000
14            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [15]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,-1.788831,-0.053289,-0.765812,-1.946661,1.383658,-1.752749,-0.621103,1.204752,-0.990234,0.166091,...,0.984445,1.435761,-0.708446,-0.446366,-0.967915,-0.373775,0.579127,1.332149,-1.644452,1.173188
1,-1.646154,0.543626,-0.732922,-1.849054,1.406794,-1.314663,-0.021470,0.820657,-0.546688,0.428911,...,-0.051710,0.887657,-1.954083,-1.451705,-1.034435,0.196789,1.104304,1.233818,-1.312944,0.756047
2,-1.097922,1.587678,0.792736,-0.976209,1.377939,-0.916498,0.340864,0.275392,0.158672,0.847995,...,-0.115145,0.582703,-1.416611,-0.616063,-1.641666,-0.838463,0.820866,0.880425,-0.996740,0.480739
3,-0.604178,1.410733,1.609275,-0.352824,1.367508,-0.556764,0.470555,-0.132596,0.824346,1.230291,...,0.633403,-0.191684,-0.777213,0.598353,-1.333696,-1.118619,0.311667,0.120113,-0.696766,0.344683
4,-0.161412,1.129186,1.550236,0.121740,-1.215809,-0.426008,0.312334,-0.104860,1.148829,1.382743,...,-0.646476,-0.738636,-0.295440,1.379072,-0.557774,-0.252116,0.316058,-1.003809,-0.491049,0.274475
5,0.228858,0.451761,1.095422,0.495478,-0.874960,0.230528,0.055613,0.657046,0.830615,1.112297,...,-0.757614,-0.094441,0.186945,1.509990,0.430797,1.205911,0.637615,-1.836837,-0.071805,0.592030
6,0.541016,0.009964,0.555991,0.682235,-0.616655,0.459321,-0.472188,1.615453,-0.215960,0.282052,...,0.345772,0.550099,0.132453,-0.927914,0.882911,2.004971,0.902142,-1.530670,0.157572,0.819692
7,0.739557,-0.413904,-0.920302,0.605403,-0.392541,0.385546,-1.222320,0.558230,-0.940033,-0.632425,...,1.298413,0.371448,0.537815,-1.566949,0.473308,0.970972,0.414308,-0.663324,0.260600,1.037237
8,0.901552,-1.031862,-1.275923,0.681176,-0.387815,1.733415,2.155314,-0.835563,-1.089341,-1.093812,...,1.596124,0.621735,1.045202,0.057796,0.515145,0.021497,-0.449746,0.006727,1.190164,-1.494981
9,1.004993,-1.415041,-0.623358,0.883876,-0.604855,1.359405,1.108906,-1.495939,-1.153056,-1.335633,...,-0.048438,0.357181,0.947123,-0.157727,1.233540,0.399746,-0.842785,0.332330,1.151189,-1.277683


In [16]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3466.227000
5     3246.749286
6     3017.084229
7     2776.217486
8     2891.489000
9     2976.485000
10    3258.027000
11    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
12,0.569016,-0.799091,-1.589659,0.368609,-0.769661,-1.10113,-2.524529,1.108405,2.635823,-0.189959,...,-2.260447,-1.929808,0.722827,0.501119,-0.180244,-1.944793,-1.604836,0.135558,0.9946,-1.378467


In [18]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [19]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1845115904, 1093860617, 2612040539, 1963713673, 2175328310, 2667129661, 1868632917, 965673439, 1761534900, 581037671, 3287987218, 3929585169, 308605944, 985494589, 2813385599, 2806634805, 3581288348, 1412571696, 1355316743, 1702098180, 3831758634, 3789607315, 3101737157, 3947662663, 1174116169, 808064420, 1350325184, 292349151, 3481323837, 790026593, 1737559044, 3920690034, 917012631, 43621482, 3425475108, 709511569, 384426670, 2234753031, 303609808, 2452348428, 2627944240, 217773465, 3391095282, 28604365, 3655654518, 203933022, 3429819978, 424989580, 2373840849, 1529774398, 1519091589, 950845608, 824757885, 2462086208, 3048342411, 2471297294, 2283627634, 3301232, 2750431037, 853373944, 639971207, 289993089, 796012918, 3982896409, 1334293491, 2958447675, 4123103966, 2817169887, 1672577887, 3748592926, 2927620552, 2877610040, 3277733743, 3811163989, 2342859227, 1525179224, 1737160690, 913414661, 1661289701, 3509659907, 4019123124, 1993669540, 4047144352, 2590313302, 2773402210, 6732575

2023-09-15 19:23:29.882040: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:23:30.429773: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:23:30.450454: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 153250.265625


Step: 17 ___________________________________________
val_loss: 54528.63671875


Step: 18 ___________________________________________
val_loss: 126156.9453125


Step: 19 ___________________________________________
val_loss: 108092.296875


Step: 20 ___________________________________________
val_loss: 115694.6484375


Step: 21 ___________________________________________
val_loss: 58788.015625


Step: 22 ___________________________________________
val_loss: 67680.890625


Step: 23 ___________________________________________
val_loss: 109421.5625


Step: 24 ___________________________________________
val_loss: 179877.875


Step: 25 ___________________________________________
val_loss: 124753.828125


Step: 26 ___________________________________________
val_loss: 209504.890625


Step: 27 ___________________________________________
val_loss: 178071.015625


Step: 28 ___________________________________________
val_loss: 83218.3046875


Step: 29 _____________________

2023-09-15 19:42:40.523631: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:42:40.548133: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:42:41.172789: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 67292.7421875


Step: 76 ___________________________________________
val_loss: 198263.875


Step: 77 ___________________________________________
val_loss: 67962.171875


Step: 78 ___________________________________________
val_loss: 146714.8125


Step: 79 ___________________________________________
val_loss: 159056.28125


Step: 80 ___________________________________________
val_loss: 107696.140625


Step: 81 ___________________________________________
val_loss: 197097.40625


Step: 82 ___________________________________________
val_loss: 194168.671875


Step: 83 ___________________________________________
val_loss: 159569.046875


Step: 84 ___________________________________________
val_loss: 111086.96875


Step: 85 ___________________________________________
val_loss: 83338.640625


Step: 86 ___________________________________________
val_loss: 68522.140625


Step: 87 ___________________________________________
val_loss: 116506.21875


Step: 88 _____________________________

In [23]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 1s 868ms/step - loss: 9598916.0000 - val_loss: 12129804.0000
Epoch 2/10000
1/1 [==============================] - 0s 16ms/step - loss: 9312550.0000 - val_loss: 8451303.0000
Epoch 3/10000
1/1 [==============================] - 0s 19ms/step - loss: 11300247.0000 - val_loss: 17726736.0000
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 8762521.0000 - val_loss: 14276424.0000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 7933625.5000 - val_loss: 16471492.0000
Epoch 6/10000
1/1 [==============================] - 0s 20ms/step - loss: 7856986.0000 - val_loss: 10899214.0000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 8150970.5000 - val_loss: 13831282.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 7877064.5000 - val_loss: 14925512.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 7542580.0000 - val_lo

1/1 [==============================] - 0s 15ms/step - loss: 2383265.7500 - val_loss: 3421798.5000
Epoch 74/10000
1/1 [==============================] - 0s 16ms/step - loss: 2091876.6250 - val_loss: 2524420.5000
Epoch 75/10000
1/1 [==============================] - 0s 16ms/step - loss: 2179307.0000 - val_loss: 2194760.5000
Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 2247259.2500 - val_loss: 2892464.5000
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 2282159.7500 - val_loss: 2096112.2500
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 2074784.8750 - val_loss: 2392019.5000
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 1860089.6250 - val_loss: 2420340.2500
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 1918981.0000 - val_loss: 1871761.3750
Epoch 81/10000
1/1 [==============================] - 0s 16ms/step - loss: 1983376.1250 - val_loss: 2245520.500

1/1 [==============================] - 0s 14ms/step - loss: 729595.7500 - val_loss: 756045.1250
Epoch 146/10000
1/1 [==============================] - 0s 15ms/step - loss: 637994.8125 - val_loss: 766855.5625
Epoch 147/10000
1/1 [==============================] - 0s 15ms/step - loss: 565941.4375 - val_loss: 713677.2500
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 584016.4375 - val_loss: 754071.3125
Epoch 149/10000
1/1 [==============================] - 0s 15ms/step - loss: 543247.1875 - val_loss: 505078.1250
Epoch 150/10000
1/1 [==============================] - 0s 16ms/step - loss: 622145.4375 - val_loss: 559987.3125
Epoch 151/10000
1/1 [==============================] - 0s 15ms/step - loss: 512211.9062 - val_loss: 520046.2812
Epoch 152/10000
1/1 [==============================] - 0s 15ms/step - loss: 503026.7500 - val_loss: 479102.2500
Epoch 153/10000
1/1 [==============================] - 0s 15ms/step - loss: 596336.5000 - val_loss: 234769.3438
Epoch 15

1/1 [==============================] - 0s 16ms/step - loss: 197866.1719 - val_loss: 393129.6250
Epoch 219/10000
1/1 [==============================] - 0s 16ms/step - loss: 217425.8281 - val_loss: 389802.6875
Epoch 220/10000
1/1 [==============================] - 0s 16ms/step - loss: 196052.6250 - val_loss: 401674.9688
Epoch 221/10000
1/1 [==============================] - 0s 17ms/step - loss: 227633.9375 - val_loss: 291078.4375
Epoch 222/10000
1/1 [==============================] - 0s 16ms/step - loss: 226812.0000 - val_loss: 471385.8438
Epoch 223/10000
1/1 [==============================] - 0s 16ms/step - loss: 228088.8750 - val_loss: 396117.8125
Epoch 224/10000
1/1 [==============================] - 0s 16ms/step - loss: 228256.2344 - val_loss: 370464.5000
Epoch 225/10000
1/1 [==============================] - 0s 16ms/step - loss: 177632.8750 - val_loss: 389476.2500
Epoch 226/10000
1/1 [==============================] - 0s 16ms/step - loss: 198991.8750 - val_loss: 391239.2500
Epoch 22

1/1 [==============================] - 0s 15ms/step - loss: 117849.1250 - val_loss: 179481.7188
Epoch 292/10000
1/1 [==============================] - 0s 15ms/step - loss: 107879.0547 - val_loss: 180068.2344
Epoch 293/10000
1/1 [==============================] - 0s 16ms/step - loss: 89261.7500 - val_loss: 162829.9844
Epoch 294/10000
1/1 [==============================] - 0s 19ms/step - loss: 91142.6875 - val_loss: 157794.9531
Epoch 295/10000
1/1 [==============================] - 0s 18ms/step - loss: 95629.3750 - val_loss: 159074.0000
Epoch 296/10000
1/1 [==============================] - 0s 15ms/step - loss: 90072.1484 - val_loss: 141167.5156
Epoch 297/10000
1/1 [==============================] - 0s 15ms/step - loss: 93875.7891 - val_loss: 192397.0312
Epoch 298/10000
1/1 [==============================] - 0s 15ms/step - loss: 91746.3438 - val_loss: 126186.0859
Epoch 299/10000
1/1 [==============================] - 0s 16ms/step - loss: 78355.0078 - val_loss: 141401.5156
Epoch 300/10000

...

1/1 [==============================] - 0s 15ms/step - loss: 58840.8242 - val_loss: 74275.8125
Epoch 810/10000
1/1 [==============================] - 0s 15ms/step - loss: 58790.2812 - val_loss: 75069.9766
Epoch 811/10000
1/1 [==============================] - 0s 15ms/step - loss: 66498.6172 - val_loss: 54972.9609
Epoch 812/10000
1/1 [==============================] - 0s 15ms/step - loss: 64675.4805 - val_loss: 74188.2734
Epoch 813/10000
1/1 [==============================] - 0s 15ms/step - loss: 71813.5938 - val_loss: 82693.3125
Epoch 814/10000
1/1 [==============================] - 0s 15ms/step - loss: 61868.9531 - val_loss: 73908.4453
Epoch 815/10000
1/1 [==============================] - 0s 20ms/step - loss: 58598.1445 - val_loss: 71626.9297
Epoch 816/10000
1/1 [==============================] - 0s 16ms/step - loss: 55056.3125 - val_loss: 65067.3398
Epoch 817/10000
1/1 [==============================] - 0s 15ms/step - loss: 61394.1367 - val_loss: 70449.9375
Epoch 818/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 59767.2930 - val_loss: 66884.8672
Epoch 884/10000
1/1 [==============================] - 0s 16ms/step - loss: 67777.8047 - val_loss: 44334.2031
Epoch 885/10000
1/1 [==============================] - 0s 16ms/step - loss: 77846.1797 - val_loss: 64509.9219
Epoch 886/10000
1/1 [==============================] - 0s 15ms/step - loss: 58474.7188 - val_loss: 49462.3672
Epoch 887/10000
1/1 [==============================] - 0s 16ms/step - loss: 64064.1562 - val_loss: 69586.2188
Epoch 888/10000
1/1 [==============================] - 0s 15ms/step - loss: 56090.9102 - val_loss: 67274.5078
Epoch 889/10000
1/1 [==============================] - 0s 16ms/step - loss: 68870.0156 - val_loss: 61878.5078
Epoch 890/10000
1/1 [==============================] - 0s 15ms/step - loss: 61446.0312 - val_loss: 59253.1406
Epoch 891/10000
1/1 [==============================] - 0s 15ms/step - loss: 62201.1289 - val_loss: 54794.0547
Epoch 892/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 64293.3945 - val_loss: 60956.4258
Epoch 958/10000
1/1 [==============================] - 0s 19ms/step - loss: 70679.6797 - val_loss: 68168.9297
Epoch 959/10000
1/1 [==============================] - 0s 15ms/step - loss: 65479.0938 - val_loss: 65753.6797
Epoch 960/10000
1/1 [==============================] - 0s 15ms/step - loss: 58701.9570 - val_loss: 63703.9844
Epoch 961/10000
1/1 [==============================] - 0s 15ms/step - loss: 62170.8320 - val_loss: 66365.8594
Epoch 962/10000
1/1 [==============================] - 0s 15ms/step - loss: 59268.4883 - val_loss: 68879.9766
Epoch 963/10000
1/1 [==============================] - 0s 15ms/step - loss: 79670.7109 - val_loss: 59006.1758
Epoch 964/10000
1/1 [==============================] - 0s 15ms/step - loss: 68399.3281 - val_loss: 71357.0703
Epoch 965/10000
1/1 [==============================] - 0s 15ms/step - loss: 51772.6680 - val_loss: 63503.1406
Epoch 966/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 66726.2266 - val_loss: 84980.2188
Epoch 1032/10000
1/1 [==============================] - 0s 15ms/step - loss: 62892.8633 - val_loss: 89560.9375
Epoch 1033/10000
1/1 [==============================] - 0s 16ms/step - loss: 62990.9727 - val_loss: 88225.3906
Epoch 1034/10000
1/1 [==============================] - 0s 15ms/step - loss: 60771.2070 - val_loss: 89921.2109
Epoch 1035/10000
1/1 [==============================] - 0s 15ms/step - loss: 60675.6055 - val_loss: 86936.1484
Epoch 1036/10000
1/1 [==============================] - 0s 19ms/step - loss: 66208.0234 - val_loss: 90745.2344
Epoch 1037/10000
1/1 [==============================] - 0s 16ms/step - loss: 59549.9062 - val_loss: 70680.1719
Epoch 1038/10000
1/1 [==============================] - 0s 15ms/step - loss: 62663.1602 - val_loss: 79732.6094
Epoch 1039/10000
1/1 [==============================] - 0s 15ms/step - loss: 59933.5195 - val_loss: 96626.1172
Epoch 1040/10000
1

Epoch 1105/10000
1/1 [==============================] - 0s 15ms/step - loss: 64449.2148 - val_loss: 67527.9062
Epoch 1106/10000
1/1 [==============================] - 0s 15ms/step - loss: 64343.8906 - val_loss: 82373.4844
Epoch 1107/10000
1/1 [==============================] - 0s 27ms/step - loss: 66939.5078 - val_loss: 88358.2891
Epoch 1107: early stopping


In [24]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 54ms/step
[4089.333](test_target) - [[3384.289]](prediction) = 705.0439375000001


In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,-0.932986,-1.099975,-0.729604,-0.816011,-0.465381,-1.243762,-1.312210,1.148205,-1.122043,-1.121418,...,1.412346,1.323311,1.276699,0.891356,0.813979,-0.083359,-1.191078,0.945270,-1.234276,1.299312
1,-0.453916,-0.219779,-0.684368,-0.592292,1.389222,0.038963,0.199402,0.140893,-0.184474,-0.185492,...,-0.643250,-0.229640,-1.165161,-1.396527,0.594549,1.264295,1.255842,0.438326,0.019291,-0.166085
2,1.386902,1.319754,1.413972,1.408303,-0.923841,1.204798,1.112809,-1.289098,1.306518,1.306910,...,-0.769096,-1.093670,-0.111538,0.505170,-1.408528,-1.180936,-0.064763,-1.383595,1.214985,-1.133227


train_target:


,Santa Catarina - Consumo de Cimento (t)
0,2669.400
1,2852.759
2,3087.810


test_input:


...

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
12,0.569016,-0.799091,-1.589659,0.368609,-0.769661,-1.10113,-2.524529,1.108405,2.635823,-0.189959,...,-2.260447,-1.929808,0.722827,0.501119,-0.180244,-1.944793,-1.604836,0.135558,0.9946,-1.378467


test_target:


,Santa Catarina - Consumo de Cimento (t)
12,4089.333


1/1 [==============================] - 0s 35ms/step
Error: 705.0439375000001




[3087.7919921875,
 3249.1259765625,
 3430.23291015625,
 3305.466552734375,
 3017.104736328125,
 2820.03271484375,
 2893.09912109375,
 2975.0791015625,
 3020.2177734375,
 3384.2890625]

In [27]:
display(targets)
display(predictions)

[3268.31,
 3466.227,
 3246.749285714286,
 3017.084228571429,
 2776.217485714286,
 2891.489,
 2976.485,
 3258.027,
 3537.032,
 4089.333]

[3087.7919921875,
 3249.1259765625,
 3430.23291015625,
 3305.466552734375,
 3017.104736328125,
 2820.03271484375,
 2893.09912109375,
 2975.0791015625,
 3020.2177734375,
 3384.2890625]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

277.00204570312496

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.08516077026552347